In [8]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

In [9]:
from typing import Annotated
from langchain_core.tools import InjectedToolCallId
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import InjectedState, create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.types import Command

class CustomState(AgentState):
    user_name: str

def update_user_info(
        tool_call_id: Annotated[str, InjectedToolCallId],
        config: RunnableConfig
) -> Command:
    """Look up and udpate user info."""
    user_id = config["configurable"].get("user_id")
    name = "Nheo" if user_id == "user_123" else "Unknown"
    return Command(update={ 
        "user_name":name,
        "messages": [ToolMessage("Successfully looked up user info", tool_call_id=tool_call_id)],}
    )

def greet(state: Annotated[CustomState, InjectedState]) -> str:
    """Use this to greet the user once you found their info."""
    user_name = state["user_name"]
    return f"Howdy {user_name}, how can I help you today?"

In [10]:
agent = create_react_agent(
    model = ChatOpenAI(temperature=0.0, model="gpt-4o"),
    tools = [update_user_info, greet],
    state_schema=CustomState
)

In [11]:
agent.invoke(
    {"messages": [{"role": "user", "content": "greet the user"}]},
    config={"configurable": {"user_id": "user_123"}}
)

{'messages': [HumanMessage(content='greet the user', additional_kwargs={}, response_metadata={}, id='85b84361-ad80-4a65-96ce-ccc476ddbdc4'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wXzSDF9glDA3ZvJ9Np2zFPaw', 'function': {'arguments': '{}', 'name': 'update_user_info'}, 'type': 'function'}, {'id': 'call_zOxN2D3xS9ARQDAvek1Vi5q9', 'function': {'arguments': '{}', 'name': 'greet'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 63, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BpVwKtQjnUf4UvFam9onziafCBt0z', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d09fc955-a16f-4ea1-ad69-46b7aca89